Goal: Create a RL Algo which can 'solve' the CartPole Game Environment

So Far, I have implemented exploration decay (epsilon decay), increasing batch size, Prioritized experience replay using recency as rank (I would like to change this to TD error ranking). This is in essence improving the weak spots

Other things to try: Double DQN, Dueling DQN, SARSA, PPO (proximal policy optimization), Policy Gradient, Natural Policy Gradient, KL-Divergence, Target Network

Open Questions: Batch size Modulation/Constant, Learning Rate Modulation, Epsilon Rate Modulation/Constant, online vs batch training

In [1]:
import gym
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


from keras.models import model_from_json
import os
from collections import deque
import itertools
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv1D
from keras.optimizers import Adam
import random
from keras.models import load_model
from keras import backend as K

Using TensorFlow backend.


In [ ]:
LEARNING_RATE = 0.0001
env = gym.make("CartPole-v0")
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n
# This method seems to overtrain so that only one action (1) is predicted
model = Sequential()

model.add(Dense(12, input_shape = (1,observation_space,), activation="relu"))
model.add(BatchNormalization())
#model.add(Dropout(rate = 0.2))
model.add(Dense(32, activation = 'relu'))
model.add(BatchNormalization())
#model.add(Dropout(rate = 0.2))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(action_space,activation = 'linear'))
model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

In [32]:
LEARNING_RATE = 0.045
env = gym.make("CartPole-v0")
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n
# This method seems to overtrain so that only one action (1) is predicted
model1 = Sequential()

model1.add(Dense(12, input_shape = (1,observation_space,), activation="relu"))
model1.add(BatchNormalization())
#model.add(Dropout(rate = 0.2))
model1.add(Dense(32, activation = 'relu'))
model1.add(BatchNormalization())
#model.add(Dropout(rate = 0.2))
model1.add(Dense(12, activation = 'relu'))
model1.add(Dense(action_space,activation = 'linear'))
model1.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))
model3 = model1

In [ ]:
LEARNING_RATE = 0.045
env = gym.make("CartPole-v0")
observation_space = env.observation_space.shape[0]
action_space = env.action_space.n
# This method seems to overtrain so that only one action (1) is predicted
model3 = Sequential()

model3.add(Dense(12, input_shape = (1,observation_space,), activation="relu"))
model3.add(BatchNormalization())
#model.add(Dropout(rate = 0.2))
model3.add(Dense(32, activation = 'relu'))
model3.add(BatchNormalization())
#model.add(Dropout(rate = 0.2))
model3.add(Dense(12, activation = 'relu'))
model3.add(Dense(action_space,activation = 'linear'))
model3.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

In [ ]:
game_count1 = 0
action_space = env.action_space.n
observation_space = env.observation_space.shape[0]
#print(observation_space)

state = env.reset()
memory1 = deque(maxlen = 1000000)
#obs = env.step(env.action_space.sample())
s = 0
save = 0
for _ in range(100000):
    state = np.reshape(state, [1,1,observation_space])
    action = np.argmax(model1.predict(state)[0][0])
    #r = (100)/(game_count+1)
    r = -(1/(6000**2))*(game_count1**2)+1
    #r = (1-((game_count1+1)/(game_count1+100)))
    #r = 0.99**(game_count1)
    r = np.clip(r,0.02,1)
    # r could also be (game_count+1)/(game_count+100)

    if np.random.uniform(0,1) < r :
        action = env.action_space.sample()
    #env.render()
    new_state, reward, done, info = env.step(action)
    
    new_state = np.reshape(new_state, [1,1,observation_space])
    memory1.append([new_state,state,reward,action])
    state = new_state
    #print(memory1)
    memory_len = len(memory1)
    if ((memory_len >= 20)):
        
        rank = np.arange(memory_len,0,-1)
        #print(rank)
        rank = 1/(rank**0.6)
        #rank = rank**0.6
        #print(rank)
        dist = rank/np.sum(rank)
        
        #print(memory_len)
        cum_dist = np.cumsum(dist)
        '''
        cum_dist = [0]*memory_len
        #print(dist)
        #print(cum_dist)
        cum_dist[0] = dist[0]
        for i in range(1,len(cum_dist)):
            #print(cum_dist[i-1])
            #print(dist[i])
            cum_dist[i] = cum_dist[i-1] + dist[i]
        #print(cum_dist)
        '''
        p = int(np.ceil(memory_len**(2/np.pi)))
        p = np.clip(p,1,20)
        p = int(p)
        
        unif = np.random.uniform(0,1,size = p)
        choices = []
        for u in range(p):
            choi = 0 
            for j in range(p):
                if unif[u] > cum_dist[j]:
                    choi += 1
                else:
                    break
            choices.append(choi)
        #print(choices)    
        temp_mem = [memory1[i] for i in choices]
        #for q in choices:
        #    temp_mem.append(memory1[q])
        #plt.bar(x = range(0,len(dist)), height = dist)
        #plt.show()
        #temp_mem = random.sample(memory1,20)
        q_val_list = deque(maxlen = p)
        #states = deque(maxlen = int(p))
        #_val_list = []
        states = [i[1] for i in temp_mem]
        #print(states)
        #rint(temp_mem)
        
        #model1.save('my_model3.h5')
        
        for new_state, state, reward, action in temp_mem:
            
            if done == True:
                update = reward
            else:
                update = reward + 0.95*(np.amax(model3.predict(new_state)[0][0]))
            q_vals = model3.predict(state)[0][0]
            q_vals[action] = update
            q_vals = np.reshape(q_vals,(1,1,2))
            q_val_list.append(q_vals)
            #states.append(state)
            #model1.fit(state,q_vals,verbose = 0)
            #print(q_vals)
        #print(np.array(states))
        #states = np.reshape(np.array(states),[20,1,4])
        #states = np.array([item for sublist in states for item in sublist])
        states = np.reshape(states, (p,1,4))
        q_val_list = np.reshape(q_val_list, (p,1,2))
        #print(states)
        #print(q_val_list)
        #print('_____________')
        #states = temp_mem[]
        model1.fit(states,q_val_list,epochs = len(temp_mem),verbose = 0)
        
    if done == True:
        s += 1
        game_count1 += 1
        #print(_-save)
        K.clear_session()
        model1 = load_model('my_model3.h5')
        model3 = model1

        print('Game Count: ' + str(game_count1) + ', length of game: ' + str(_-save))
        save = _
        state = env.reset()
        continue

Game Count: 1, length of game: 24
Game Count: 2, length of game: 10
Game Count: 3, length of game: 26
Game Count: 4, length of game: 20
Game Count: 5, length of game: 38
Game Count: 6, length of game: 11
Game Count: 7, length of game: 20
Game Count: 8, length of game: 19
Game Count: 9, length of game: 74
Game Count: 10, length of game: 27
Game Count: 11, length of game: 74
Game Count: 12, length of game: 19
Game Count: 13, length of game: 28
Game Count: 14, length of game: 17
Game Count: 15, length of game: 36
Game Count: 16, length of game: 13
Game Count: 17, length of game: 16
Game Count: 18, length of game: 19
Game Count: 19, length of game: 21
Game Count: 20, length of game: 12
Game Count: 21, length of game: 30
Game Count: 22, length of game: 20
Game Count: 23, length of game: 22
Game Count: 24, length of game: 24
Game Count: 25, length of game: 13
Game Count: 26, length of game: 27
Game Count: 27, length of game: 16
Game Count: 28, length of game: 22
Game Count: 29, length of gam

In [ ]:
#Continuous train!
s = 0
#memory = deque(maxlen = 1000000)
save = 0
for _ in range(100000):
    state = np.reshape(state, [1,1,observation_space])
    action = np.argmax(model1.predict(state)[0])
     #r = (100)/(game_count+1)
    r = -(1/(6000**2))*(game_count1**2)+1
    #r = (1-((game_count1+1)/(game_count1+100)))
    #r = 0.99**(game_count1)
    r = np.clip(r,0,0.98)
    # r could also be (game_count+1)/(game_count+100)
    if np.random.uniform(0,1) < r :
        action = env.action_space.sample()
    #env.render()
    new_state, reward, done, info = env.step(action)
   
    new_state = np.reshape(new_state, [1,1, observation_space])
    memory1.append((new_state,state,reward,action))
    state = new_state
    
    if ((len(memory1) >= 2) & (_%4 == 0)):
        rank = np.arange(len(memory1)-1,0,-1)
        #print(rank)
        rank = 1/rank
        #print(rank)
        total = np.sum(rank**0.6)
        numerator = rank**0.6
        dist = numerator/total
        
        
        cum_dist = [0]*len(dist)
        cum_dist[0] = dist[0]
        for i in range(1,len(dist)):
            cum_dist[i] = cum_dist[i-1] + dist[i]
        #print(cum_dist)
        unif = np.random.uniform(0,1,size = int(np.ceil(len(memory1)**(2/np.pi))))
        choices = []
        for u in range(len(unif)):
            choi = 0 
            for j in range(len(cum_dist)):
                if unif[u] > cum_dist[j]:
                    choi += 1
            choices.append(choi)
        #print(choices)    
        temp_mem = np.array(memory1)[choices]
        #for q in choices:
        #    temp_mem.append(memory1[q])
        #plt.bar(x = range(0,len(dist)), height = dist)
        #plt.show()
        #temp_mem = random.sample(memory1,20)
        
        
        q_val_list = []
        states = []
        
        for new_state, state, reward, action in temp_mem:
            update = reward + 0.95*(np.amax(model2.predict(new_state)[0][0]))
            if done == True:
                update = reward
            q_vals = model1.predict(state)[0][0]
            q_vals[action] = update
            q_vals = np.reshape(q_vals,(1,1,2))
            #q_val_list.append(q_vals)
            #states.append(state)
            model1.fit(state,q_vals,verbose = 0)
            #print(q_vals)
        #print(np.array(states))
        #states = np.reshape(np.array(states),[20,1,4])
        #states = np.array([item for sublist in states for item in sublist])
        #states = np.reshape(states, (len(temp_mem),1,4))
        #q_val_list = np.reshape(q_val_list, (len(temp_mem),1,2))
        #print(states)
        #print(q_val_list)
        #print('_____________')
        #states = temp_mem[]
        #model1.fit(states,q_val_list,verbose = 0)
        if done == True:
            s += 1
            game_count1 += 1
            '''
            model_json = model1.to_json()
            with open("model1.json", "w") as json_file:
                json_file.write(model_json)
            # serialize weights to HDF5
            model1.save_weights("model1.h5")
            print("Saved model to disk")
            '''
            print('Game Count: ' + str(game_count1) + ', length of game: ' + str(_-save))
            save = _
            if game_count1 == 10000:
                break
            state = env.reset()
            continue

In [ ]:
state = env.reset()
#game_state_save = game_count1
#game_count1 = 0
#memory = deque(maxlen = 1000000)
#obs = env.step(env.action_space.sample())
save = 0
lens = []
for _ in range(300):
    state = np.reshape(state, [1,1, observation_space])
    action = np.argmax(model1.predict(state)[0])
    #print(model1.predict(state))
    #print(action)
    
    env.render()
    new_state, reward, done, info = env.step(action)
    if done == True:
        #print(_-save)
        lens.append(_-save)
        save = _
        
        #env.close()
        state = env.reset()
        continue
    new_state = np.reshape(new_state, [1,1, observation_space])
    #memory.append((new_state,state,reward,action))
    state = new_state
env.close()
plt.hist(lens)
plt.show()
print(np.max(lens),np.std(lens),np.median(lens),np.mean(lens))
#game_count1 = game_state_save

In [ ]:
game_count = 0
action_space = env.action_space.n
observation_space = env.observation_space.shape[0]
#print(observation_space)

state = env.reset()
memory = deque(maxlen = 1000000)
#obs = env.step(env.action_space.sample())
s = 0
for _ in range(100000):
    state = np.reshape(state, [1,1,observation_space])
    action = np.argmax(model.predict(state)[0][0])
    if np.random.uniform(0,1) > 0.9:
        action = env.action_space.sample()
    #env.render()
    new_state, reward, done, info = env.step(action)
    if done == True:
        s += 1
        game_count += 1
        print(game_count)
        state = env.reset()
        continue
    new_state = np.reshape(new_state, [1,1,observation_space])
    memory.append((new_state,state,reward,action))
    state = new_state
    if len(memory) >= 20:
        temp_mem = random.sample(memory,20)
        q_val_list = []
        states = []
        for new_state, state, reward, action in temp_mem:
            update = reward + 0.95*(np.amax(model.predict(new_state)[0][0]))
            q_vals = model.predict(state)[0][0]
            q_vals[action] = update
            q_vals = np.reshape(q_vals,(1,1,2))
            q_val_list.append(q_vals)
            states.append(state)
            model.fit(state,q_vals,verbose = 0)
            #print(q_vals)
        #print(np.array(states))
        #states = np.reshape(np.array(states),[20,1,4])
        #states = np.array([item for sublist in states for item in sublist])
        #states = np.reshape(states, (20,1,4))
        #q_val_list = np.reshape(q_val_list, (20,1,2))
        #print(states)
        #print(q_val_list)
        #print('_____________')
        #states = temp_mem[]
        #model.fit(states,q_val_list,verbose = 0)

In [ ]:
#Continuous train!
s = 0
#memory = deque(maxlen = 1000000)
for _ in range(100000):
    state = np.reshape(state, [1,1,observation_space])
    action = np.argmax(model.predict(state)[0])
    if np.random.uniform(0,1) > (game_count+1)/(game_count+100):
        action = env.action_space.sample()
    #env.render()
    new_state, reward, done, info = env.step(action)
    if done == True:
        s += 1
        game_count += 1
        print(game_count)
        state = env.reset()
        continue
    new_state = np.reshape(new_state, [1,1, observation_space])
    memory.append((new_state,state,reward,action))
    state = new_state
    if len(memory) >= 20:
        temp_mem = random.sample(memory,20)
        q_val_list = []
        states = []
        for new_state, state, reward, action in temp_mem:
            update = reward + 0.95*(np.amax(model.predict(new_state)[0][0]))
            q_vals = model.predict(state)[0][0]
            q_vals[action] = update
            q_vals = np.reshape(q_vals,(1,1,2))
            q_val_list.append(q_vals)
            states.append(state)
            model.fit(state,q_vals,verbose = 0)
            #print(q_vals)
        #print(np.array(states))
        #states = np.reshape(np.array(states),[20,1,4])
        #states = np.array([item for sublist in states for item in sublist])
        #states = np.reshape(states, (20,1,4))
        #q_val_list = np.reshape(q_val_list, (20,1,2))
        #print(states)
        #print(q_val_list)
        #print('_____________')
        #states = temp_mem[]
        #model.fit(states,q_val_list,verbose = 0)

In [ ]:
state = env.reset()
#memory = deque(maxlen = 1000000)
#obs = env.step(env.action_space.sample())

for _ in range(1000):
    state = np.reshape(state, [1,1, observation_space])
    action = np.argmax(model.predict(state)[0])
    print(model.predict(state))
    print(action)
    env.render()
    new_state, reward, done, info = env.step(action)
    if done == True:
        #print(_)
        #env.close()
        state = env.reset()
        continue
    new_state = np.reshape(new_state, [1,1, observation_space])
    #memory.append((new_state,state,reward,action))
    state = new_state
env.close()

In [ ]:
env.close()

In [ ]:
observation_space

In [ ]:
obs